# Small svd optimization - debugging

Created 26/02/2025

Objectives:
* Perform simple checks on calculated spt states.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

from tenpy.models.model import CouplingMPOModel
from tenpy.networks.site import ClockSite, FermionSite

import os

In [3]:
from functools import reduce
from operator import mul
from itertools import product

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.Optimizers.OneSiteSolver import OneSiteSolver

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    get_left_identity_environment_from_tp_tensor,
    multiply_transfer_matrices
)

from SPTOptimization.Optimizers.utils import (
    one_site_optimization_sweep_right
)

from SPTOptimization.gradients import (
    expectation_gradient_from_environments_and_b_tensor
)

In [6]:
import re

# Load data

In [7]:
DATA_DIR = r"../data/simple_fermionic_cluster_200_site_dmrg/"

In [8]:
def parse_file_name(file_name):
    non_trivial_proj_rep = bool(int(file_name[3]))

    n1_01 = int(file_name[8])
    n1_10 = int(file_name[9])

    return (non_trivial_proj_rep, (n1_01, n1_10))

In [9]:
loaded_data = dict()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data[data_info]=data['wavefunction']

In [10]:
loaded_data

{(False, (1, 1)): <tenpy.networks.mps.MPS at 0x13870aa90>,
 (True, (1, 1)): <tenpy.networks.mps.MPS at 0x16826eb50>,
 (True, (0, 1)): <tenpy.networks.mps.MPS at 0x1682f7690>,
 (False, (0, 1)): <tenpy.networks.mps.MPS at 0x168d81f90>,
 (False, (1, 0)): <tenpy.networks.mps.MPS at 0x168ddf6d0>,
 (True, (1, 0)): <tenpy.networks.mps.MPS at 0x1672cabd0>,
 (True, (0, 0)): <tenpy.networks.mps.MPS at 0x1698c9110>,
 (False, (0, 0)): <tenpy.networks.mps.MPS at 0x168de7ed0>}

In [11]:
loaded_data[(True, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 4.10310779e-55, 3.20108804e-32, 4.41166527e-30,
       4.30652209e-31, 1.28028128e-32, 2.18979328e-32, 9.64766348e-32,
       6.39204896e-31, 4.19476267e-32, 7.51835939e-33, 1.56998498e-32,
       3.06132263e-29, 6.24832887e-32, 6.64529320e-32, 6.61192056e-33,
       3.02331734e-32, 1.97077344e-32, 3.97848871e-32, 1.65936555e-32,
       8.12962747e-32, 8.36180681e-32, 1.05455111e-32, 2.30695596e-32,
       4.15719467e-32, 3.32855567e-32, 1.02762007e-32, 1.46697913e-32,
       3.80115562e-31, 1.63756131e-32, 9.98608231e-33, 1.82981182e-32,
       1.73380216e-33, 6.00451123e-32, 1.33036658e-32, 2.79091135e-31,
       1.64291033e-32, 3.68006128e-33, 9.95622775e-32, 6.17421509e-32,
       2.70280235e-32, 3.01371615e-31, 2.35183074e-32, 1.17896355e-32,
       3.93686429e-31, 8.45721273e-32, 1.08863312e-32, 3.77353007e-31,
       9.67562785e-33, 1.99656454e-33, 6.40393616e-33, 2.78084102e-32,
       1.03150150e-31, 4.37579428e-33, 1.45445289e-32, 3.55938597e-32,
      

In [12]:
loaded_data[(False, (0,0))].expectation_value('N', sites=list(range(1, 201, 2)))

array([0.00000000e+00, 3.26915052e-29, 3.33293732e-29, 0.00000000e+00,
       3.33293732e-29, 7.79030959e-29, 4.21824880e-29, 4.21824880e-29,
       5.96576060e-30, 1.14662165e-29, 3.59424750e-29, 8.18720523e-29,
       4.21824880e-29, 8.28796989e-29, 4.21824880e-29, 3.79670126e-29,
       4.21824880e-29, 8.18720523e-29, 1.37190923e-28, 0.00000000e+00,
       1.37190923e-28, 0.00000000e+00, 3.66111579e-29, 3.66111579e-29,
       3.86541844e-29, 3.33293732e-29, 0.00000000e+00, 0.00000000e+00,
       3.59424750e-29, 1.56000325e-28, 3.46235982e-29, 0.00000000e+00,
       3.59424750e-29, 3.59424750e-29, 3.72860037e-29, 0.00000000e+00,
       0.00000000e+00, 9.22258517e-29, 8.33234331e-30, 9.01057880e-29,
       3.46235982e-29, 4.43734259e-29, 3.14342582e-29, 3.14342582e-29,
       0.00000000e+00, 8.18720523e-29, 1.46709639e-29, 1.46709639e-29,
       1.46709639e-29, 8.98561875e-30, 0.00000000e+00, 8.01495006e-30,
       4.07526776e-29, 4.00470169e-29, 9.32150093e-30, 7.69262642e-29,
      

# Definitons

In [13]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [14]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [15]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [16]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [17]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [110]:
np_c_plus = np.array([
    [0, 1],
    [0, 0]
])

np_c_minus = np.array([
    [0, 0],
    [1, 0]
])

In [18]:
fermionic_np_symmetries = [np_I, np_JW]

In [19]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [20]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [21]:
shifted_symmetry_actions = [
    [[f, b] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [22]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [23]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

## Projectors

In [24]:
def mod_4_to_bit(x):    
    possible_mod_4_values = list(range(4))
    assert x in possible_mod_4_values

    x0 = x % 2
    x1 = ((x - x0) // 2) % 2

    return (x0, x1)

In [25]:
def bit_to_mod_4(bit_pair):
    x0, x1 = bit_pair

    possible_mod_2_values = list(range(2))
    assert x0 in possible_mod_2_values
    assert x1 in possible_mod_2_values

    return x0 + 2*x1

In [26]:
def mod_4_to_bit_addition(x, y):
    (x0, x1) = mod_4_to_bit(x)
    (y0, y1) = mod_4_to_bit(y)

    out0 = (x0 + y0) % 2
    out1 = (x1 + y1) % 2

    out = bit_to_mod_4((out0, out1))

    return out

In [27]:
def get_n1_func(n1_01, n1_10):
    l = [0, n1_01, n1_10, (n1_01 + n1_10)%2]
    
    def f(g1, g2):
        i = mod_4_to_bit_addition(g1, g2)
        return l[i]

    return f

In [28]:
n1_pair_to_fermion_operators_dict = {
    (0, 0): "Id",
    (0, 1): "C",
    (1, 0): "Cd",
    (1, 1): "N"
}

In [29]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [30]:
pauli_proj_rep_phases = np.array([
    [1, 1, 1, 1],
    [1, 1, -1, -1],
    [1, -1, 1, -1],
    [1, -1, -1, 1]
])

In [31]:
def get_proj_rep_term_phase(group_quad):
    g_left, g_in, g_out, g_right = group_quad

    g1 = mod_4_to_bit_addition(g_left, g_in)
    g2 = mod_4_to_bit_addition(g_in, g_right)
    g3 = mod_4_to_bit_addition(g_left, g_out)
    g4 = mod_4_to_bit_addition(g_out, g_right)

    numerator_phase = pauli_proj_rep_phases[g1, g2]
    denominator_phase = pauli_proj_rep_phases[g3, g4]

    return numerator_phase/denominator_phase

In [32]:
def get_op_list(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    if n1_func(g_left, g_out):
        out.append(('Cd', 0, 1))
    if n1_func(g_out, g_right):
        out.append(('Cd', 1, 1))

    if n1_func(g_in, g_right):
        out.append(('C', 1, 1))
    if n1_func(g_left, g_in):
        out.append(('C', 0, 1))

    return out

In [33]:
def get_op_list_trivial(g_quad):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    out.append(('C', 1, 1))
    out.append(('C', 0, 1))

    out.append(('Cd', 0, 1))
    out.append(('Cd', 1, 1))

    return out

In [34]:
class ClusterFermion(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    # Would it be easier to subclass ClockSite?
    def init_sites(self, model_params):
        spin = ClockSite(4, conserve=None)
        for i in range(4):
            for j in range(4):
                X = np.zeros((4,4))
                X[j, i] = 1
                op_name = f"map_{i}_{j}"
                spin.add_op(op_name, X)
        ferm = FermionSite(conserve=None)
        sites = [spin, ferm]
        return [spin, ferm], ['s', 'f']

    def init_terms(self, model_params):
        # Read off model parameters
        n1_01, n1_10 = model_params.get('n1', (0,0))
        n1_func = get_n1_func(n1_01, n1_10)

        non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

        group_quads = product(range(4), repeat=4)

        for group_quad in group_quads:
            if (n1_01, n1_10) == (0, 0):
                op_list = get_op_list_trivial(group_quad)
            else:
                op_list = get_op_list(group_quad, n1_func)

            if non_trivial_proj_rep:
                phase = get_proj_rep_term_phase(group_quad)
            else:
                phase = 1

            self.add_multi_coupling((-1/4)*phase, op_list)

In [35]:
models = {
    (b, p): ClusterFermion({'L': 3, 'n1': p, 'non_trivial_proj_rep': b})
    for p in n1_pairs
    for b in [True, False]
}

In [36]:
mpos = {k: m.calc_H_MPO() for k, m in models.items()}

In [37]:
test_mpo = mpos[(False, (0,0))]

In [38]:
test_mpo.L

6

In [39]:
test_mpo.get_W(0)

<npc.Array shape=(2, 6, 4, 4) labels=['wL', 'wR', 'p', 'p*']>

In [40]:
test_mpo.get_W(5)

<npc.Array shape=(2, 2, 2, 2) labels=['wL', 'wR', 'p', 'p*']>

# Tests
## Projector test

Define rdms.

In [45]:
def get_left_environment(psi, index):
    left_leg = psi.get_B(index).legs[0]
    SL = npc.diag(psi.get_SL(index), left_leg, labels = ['vL', 'vR'])

    # Schmidt values should all be real, so conjugate not really necessary here
    left_environment = (
        npc.tensordot(SL, SL.conj(), (['vL',], ['vL*',]))
    )

    return left_environment

In [49]:
def extract_rdm(psi, left_site_index, num_sites):
    t = get_left_environment(psi, left_site_index)

    sites = range(left_site_index, left_site_index + num_sites)
    for i, site_index in enumerate(sites):
        b = psi.get_B(site_index)
        b = b.replace_label('p', f'p{i}')
        
        t = npc.tensordot(t, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b.conj(), [['vR*',], ['vL*',]])
    
    t = npc.trace(t, 'vR', 'vR*')

    return t

In [64]:
def contract_mpo_rdm(rdm, mpo):
    num_sites = mpo.L
    
    w = mpo.get_W(0).take_slice([mpo.get_IdL(0)], ['wL'])
    t = npc.tensordot(rdm, w, [['p0*', 'p0'], ['p', 'p*']])

    for i in range(1, num_sites):
        w = mpo.get_W(i)
        t = npc.tensordot(t, w, [[f'p{i}', f'p{i}*', 'wR'], ['p*', 'p', 'wL']])
    
    t = t[mpo.get_IdR(num_sites-1)]

    return t

In [65]:
left_site_index = 98
num_sites = 6

exps = dict()

iter_quads = zip(
    [True, False],
    n1_pairs,
    [True, False],
    n1_pairs
)
    

for data_pr, data_n1 in product([True, False], n1_pairs):
    psi = loaded_data[(data_pr, data_n1)]
    rdm = extract_rdm(psi, left_site_index, num_sites)
    
    
    for model_pr, model_n1 in product([True, False], n1_pairs):
        mpo = mpos[(model_pr, model_n1)]

        exp = contract_mpo_rdm(rdm, mpo)
        
        exps[(data_pr, data_n1, model_pr, model_n1)] = exp

In [66]:
exps

{(True, (0, 0), True, (0, 0)): -1.0000000000000002,
 (True, (0, 0), True, (0, 1)): -0.1249999999999977,
 (True, (0, 0), True, (1, 0)): -0.12500000000000255,
 (True, (0, 0), True, (1, 1)): -0.12499999999999759,
 (True, (0, 0), False, (0, 0)): -0.25,
 (True, (0, 0), False, (0, 1)): -0.1249999999999977,
 (True, (0, 0), False, (1, 0)): -0.12500000000000255,
 (True, (0, 0), False, (1, 1)): -0.12499999999999759,
 (True, (0, 1), True, (0, 0)): -0.1250000000000015,
 (True, (0, 1), True, (0, 1)): -1.0000000000000007,
 (True, (0, 1), True, (1, 0)): -0.2031250000000001,
 (True, (0, 1), True, (1, 1)): -0.2031249999999999,
 (True, (0, 1), False, (0, 0)): -0.12500000000000133,
 (True, (0, 1), False, (0, 1)): -0.24999999999999992,
 (True, (0, 1), False, (1, 0)): -0.20312500000000014,
 (True, (0, 1), False, (1, 1)): -0.20312499999999986,
 (True, (1, 0), True, (0, 0)): -0.12499999999999638,
 (True, (1, 0), True, (0, 1)): -0.20312500000000172,
 (True, (1, 0), True, (1, 0)): -0.9999999999999989,
 (True, 

Passes the test.

## Fermion Expectation values

In [82]:
test_psi = loaded_data[(False, (0, 1))]

In [84]:
test_psi.expectation_value_multi_sites(["map_0_0", 'N', "map_0_0"], 100)

array(3.14109075e-18)

In [103]:
fermion_exps = dict()

for data_pr, data_n1 in product([True, False], n1_pairs):
    for i_left in range(4): 
        for i_right in range(4):
            psi = loaded_data[(data_pr, data_n1)]
            exp = psi.expectation_value_multi_sites(
                [f"map_{i_left}_{i_left}", 'N', f"map_{i_right}_{i_right}"], 100
            )
            
            fermion_exps[(data_pr, data_n1, i_left, i_right)] = exp[()]

In [104]:
1/16

0.0625

In [105]:
fermion_exps

{(True, (0, 0), 0, 0): 5.447454590821024e-35,
 (True, (0, 0), 0, 1): 7.107522494590287e-34,
 (True, (0, 0), 0, 2): 1.8743101355994978e-35,
 (True, (0, 0), 0, 3): 4.922238057817101e-34,
 (True, (0, 0), 1, 0): 2.434766822599829e-36,
 (True, (0, 0), 1, 1): 2.91162669233076e-35,
 (True, (0, 0), 1, 2): 8.95438819601565e-36,
 (True, (0, 0), 1, 3): 6.964574111267624e-35,
 (True, (0, 0), 2, 0): 2.00715168843113e-34,
 (True, (0, 0), 2, 1): 2.1945365270782734e-33,
 (True, (0, 0), 2, 2): 7.291112748429983e-34,
 (True, (0, 0), 2, 3): 8.293618197159041e-35,
 (True, (0, 0), 3, 0): 6.448251497763532e-35,
 (True, (0, 0), 3, 1): 9.827481496135187e-34,
 (True, (0, 0), 3, 2): 1.3011328614695881e-34,
 (True, (0, 0), 3, 3): 6.329481910407371e-34,
 (True, (0, 1), 0, 0): -1.6182250114375938e-18,
 (True, (0, 1), 0, 1): 2.821131991091816e-18,
 (True, (0, 1), 0, 2): 0.06250000000000211,
 (True, (0, 1), 0, 3): 0.06250000000000305,
 (True, (0, 1), 1, 0): 2.94993632129426e-18,
 (True, (0, 1), 1, 1): 8.447595814589

In [107]:
[
    v
    for k, v in fermion_exps.items()
    if v > 0.01
]

[0.06250000000000211,
 0.06250000000000305,
 0.06250000000000623,
 0.06250000000000577,
 0.06249999999999793,
 0.062499999999996406,
 0.062499999999995746,
 0.06249999999999445,
 0.06249999999999688,
 0.06250000000000001,
 0.06250000000000845,
 0.062499999999999264,
 0.06249999999998945,
 0.06249999999999283,
 0.06250000000000633,
 0.06249999999999714,
 0.06249999999999945,
 0.062499999999986816,
 0.06250000000000132,
 0.06250000000000268,
 0.06250000000000133,
 0.06250000000000186,
 0.062499999999988745,
 0.062499999999976234,
 0.06250000000000797,
 0.06250000000000734,
 0.06250000000000586,
 0.06250000000000522,
 0.06249999999998972,
 0.06249999999999418,
 0.06249999999998998,
 0.06249999999999444,
 0.06249999999999719,
 0.06249999999999707,
 0.06249999999999756,
 0.06250000000000813,
 0.06249999999998738,
 0.06249999999998725,
 0.0625000000000009,
 0.06250000000001146,
 0.0624999999999997,
 0.06249999999999843,
 0.06249999999999651,
 0.062499999999999806,
 0.062499999999998286,
 0.0

In [106]:
[
    k
    for k, v in fermion_exps.items()
    if v > 0.01
]

[(True, (0, 1), 0, 2),
 (True, (0, 1), 0, 3),
 (True, (0, 1), 1, 2),
 (True, (0, 1), 1, 3),
 (True, (0, 1), 2, 0),
 (True, (0, 1), 2, 1),
 (True, (0, 1), 3, 0),
 (True, (0, 1), 3, 1),
 (True, (1, 0), 0, 1),
 (True, (1, 0), 0, 3),
 (True, (1, 0), 1, 0),
 (True, (1, 0), 1, 2),
 (True, (1, 0), 2, 1),
 (True, (1, 0), 2, 3),
 (True, (1, 0), 3, 0),
 (True, (1, 0), 3, 2),
 (True, (1, 1), 0, 1),
 (True, (1, 1), 0, 2),
 (True, (1, 1), 1, 0),
 (True, (1, 1), 1, 3),
 (True, (1, 1), 2, 0),
 (True, (1, 1), 2, 3),
 (True, (1, 1), 3, 1),
 (True, (1, 1), 3, 2),
 (False, (0, 1), 0, 2),
 (False, (0, 1), 0, 3),
 (False, (0, 1), 1, 2),
 (False, (0, 1), 1, 3),
 (False, (0, 1), 2, 0),
 (False, (0, 1), 2, 1),
 (False, (0, 1), 3, 0),
 (False, (0, 1), 3, 1),
 (False, (1, 0), 0, 1),
 (False, (1, 0), 0, 3),
 (False, (1, 0), 1, 0),
 (False, (1, 0), 1, 2),
 (False, (1, 0), 2, 1),
 (False, (1, 0), 2, 3),
 (False, (1, 0), 3, 0),
 (False, (1, 0), 3, 2),
 (False, (1, 1), 0, 1),
 (False, (1, 1), 0, 2),
 (False, (1, 1),

Looks good.

## Identity Boundary operators

In [69]:
cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*50,
                left_symmetry_index=50,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I, np_00]*1
            )

            cases[(*k, i, j)] = case

In [72]:
for c in cases.values():
    c.compute_svd_approximate_expectation()

In [78]:
right_exps = {
    k: round(np.abs(v.right_expectation), 1)
    for k, v in cases.items()
}

In [81]:
{
    k: v for k, v in right_exps.items()
    if not k[0]
}

{(False, (1, 1), 0, 0): 1.4,
 (False, (1, 1), 0, 1): 0.0,
 (False, (1, 1), 0, 2): 0.0,
 (False, (1, 1), 0, 3): 1.4,
 (False, (1, 1), 1, 0): 0.0,
 (False, (1, 1), 1, 1): 0.0,
 (False, (1, 1), 1, 2): 0.0,
 (False, (1, 1), 1, 3): 0.0,
 (False, (0, 1), 0, 0): 1.4,
 (False, (0, 1), 0, 1): 1.4,
 (False, (0, 1), 0, 2): 0.0,
 (False, (0, 1), 0, 3): 0.0,
 (False, (0, 1), 1, 0): 0.0,
 (False, (0, 1), 1, 1): 0.0,
 (False, (0, 1), 1, 2): 0.0,
 (False, (0, 1), 1, 3): 0.0,
 (False, (1, 0), 0, 0): 1.4,
 (False, (1, 0), 0, 1): 0.0,
 (False, (1, 0), 0, 2): 1.4,
 (False, (1, 0), 0, 3): 0.0,
 (False, (1, 0), 1, 0): 0.0,
 (False, (1, 0), 1, 1): 0.0,
 (False, (1, 0), 1, 2): 0.0,
 (False, (1, 0), 1, 3): 0.0,
 (False, (0, 0), 0, 0): 1.0,
 (False, (0, 0), 0, 1): 1.0,
 (False, (0, 0), 0, 2): 1.0,
 (False, (0, 0), 0, 3): 1.0,
 (False, (0, 0), 1, 0): 1.0,
 (False, (0, 0), 1, 1): 1.0,
 (False, (0, 0), 1, 2): 1.0,
 (False, (0, 0), 1, 3): 1.0}

## Theoretical solution boundary operators

In [109]:
rdm

<npc.Array shape=(4, 4, 2, 2, 4, 4, 2, 2, 4, 4, 2, 2) labels=['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*', 'p3', 'p3*', 'p4', 'p4*', 'p5', 'p5*']>

In [111]:
fermion_operators = {
    (0, 0): np_I,
    (1, 0): np_c_minus,
    (0, 1): np_c_plus,
    (1, 1): np_I
}

In [112]:
def boundary_operator_fermion_component(n1_func, g_left, g_right, g_sym):
    before_n = n1_func(g_left, g_right)
    after_n = n1_func(mod_4_to_bit_addition(g_left, g_sym), g_right)
    
    out = fermion_operators[(before_n, after_n)]

    return out

In [113]:
def np_tensor_product(X, Y):
    out = (
        X[(...,) + (np.newaxis,)*len(Y.shape)]
        *Y[(np.newaxis,)*len(X.shape) + (...,)]
    )

    return out

In [119]:
test_rdm = extract_rdm(loaded_data[(False, (0, 0))], 100, 3)

In [120]:
test_rdm

<npc.Array shape=(4, 4, 2, 2, 4, 4) labels=['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*']>

In [126]:
leg_labels = ['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*']
leg_charges = [
    test_rdm.split_legs().get_leg(l)
    for l in leg_labels
]

In [127]:
def predicted_boundary_operator(n1_func, g_sym):
    np_arrays = list()

    for g_left, g_right in product(range(4), repeat=2):
        X_left = np.zeros((4,4))
        X_left[g_left, g_left] = 1
        
        X_right = np.zeros((4,4))
        X_right[g_right, g_right] = 1
    
        X_fermion = boundary_operator_fermion_component(
            n1_func, g_left, g_right, g_sym
        )

        X = np_tensor_product(X_left, X_fermion)
        X = np_tensor_product(X, X_right)
    
        np_arrays.append(X)

    X = sum(np_arrays)
    X = X.astype(np.complex128)

    npc_X = npc.Array.from_ndarray(
        X,
        leg_charges,
        dtype=np.complex128,
        labels=leg_labels
    )

    return npc_X

In [128]:
predicted_boundary_operators = dict()

for n1_pair in n1_pairs:
    for g_sym in range(4):
        n1_func = get_n1_func(*n1_pair)
        op = predicted_boundary_operator(n1_func, g_sym)
        predicted_boundary_operators[(n1_pair, g_sym)] = op

In [130]:
predicted_boundary_operators[((0, 0), 1)]

<npc.Array shape=(4, 4, 2, 2, 4, 4) labels=['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*']>

In [136]:
test_case = list(cases.values())[0]

In [138]:
test_case.right_symmetry_index

<npc.Array shape=(2, 2) labels=['vR', 'vR*']>

In [148]:
def extract_right_rdm(case, num_sites):
    t = case.right_projected_symmetry_state
    psi = case.psi

    right_symmetry_index = test_case.right_symmetry_index + 1
    sites = range(right_symmetry_index, right_symmetry_index + num_sites)
    for i, site_index in enumerate(sites):
        b = psi.get_B(site_index)
        b = b.replace_label('p', f'p{i}')
        
        t = npc.tensordot(t, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b.conj(), [['vR*',], ['vL*',]])
    
    t = npc.trace(t, 'vR', 'vR*')

    return t

In [149]:
num_sites = 3

predicted_exps = dict()

for k, v in cases.items():
    pr, n1_pair, jw, g_sym = k

    rdm = extract_right_rdm(case, num_sites)
    op = predicted_boundary_operators[(n1_pair, g_sym)]

    exp = npc.tensordot(
        rdm,
        op,
        [
            ['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*'],
            ['p0*', 'p0', 'p1*', 'p1', 'p2*', 'p2'],
        ]
    )
    
    predicted_exps[k] = exp

In [151]:
{
    k: v
    for k, v in predicted_exps.items()
    if not k[0]
}

{(False, (1, 1), 0, 0): (0.9999999999999998+0j),
 (False, (1, 1), 0, 1): (-3.053113317719145e-15+0j),
 (False, (1, 1), 0, 2): (-3.053113317719145e-15+0j),
 (False, (1, 1), 0, 3): (0.9999999999999998+0j),
 (False, (1, 1), 1, 0): (0.9999999999999998+0j),
 (False, (1, 1), 1, 1): (-3.053113317719145e-15+0j),
 (False, (1, 1), 1, 2): (-3.053113317719145e-15+0j),
 (False, (1, 1), 1, 3): (0.9999999999999998+0j),
 (False, (0, 1), 0, 0): (0.9999999999999998+0j),
 (False, (0, 1), 0, 1): (0.9999999999999998+0j),
 (False, (0, 1), 0, 2): (-3.053113317719145e-15+0j),
 (False, (0, 1), 0, 3): (-3.053113317719145e-15+0j),
 (False, (0, 1), 1, 0): (0.9999999999999998+0j),
 (False, (0, 1), 1, 1): (0.9999999999999998+0j),
 (False, (0, 1), 1, 2): (-3.053113317719145e-15+0j),
 (False, (0, 1), 1, 3): (-3.053113317719145e-15+0j),
 (False, (1, 0), 0, 0): (0.9999999999999998+0j),
 (False, (1, 0), 0, 1): (-3.053113317719145e-15+0j),
 (False, (1, 0), 0, 2): (0.9999999999999998+0j),
 (False, (1, 0), 0, 3): (-3.05311